In [178]:
import os, sys
import re
import pandas as pd
import numpy as np

dir_path = "Files2"
ls_dir = os.listdir(dir_path)
year = 0;

# Finding Directory or year for which csv are present
for file in ls_dir:
    regexp = re.compile(r'.txt')
    print(file)
    if not(regexp.search(file)):
        year = file
    
# Setting Directory for the year    
if not(year == 0):
    print(year)
    dir_path += "/" + str(year)
else:
    print("No Files found! Ending Program")
    sys.exit()
    
#Looping over all the csv to load and process the data
ls_dir = os.listdir(dir_path)

for file in ls_dir:
#     only if file is csv
    regexp = re.compile(r'.csv')
    if(regexp.search(file)):
        print(file)
        filePath = dir_path + "/" + file
    #   Reading File data with pandas
        fileData = pd.read_csv(filePath,header = 0)
#         Printing file shape -  rows,columns
        print(fileData.shape)
#     printing colum values
        print(fileData['date'])
    
#       adding columns for year, month and day of month
        fileData = split_date(fileData)
#       adding column for true extension
        fileData = true_extention(fileData)
#       replace empty values for browser
        fileData = clean_browser(fileData, mode_of_browser)
        
        
        print(fileData['browser'])
        break
        
# Adding    
# rows_with_missing_data = all_file_data[pd.isnull(all_file_data['code'])]
# print(rows_with_missing_data)



2006
ex.txt
2006
log20060101.csv
(4461, 15)
0       2006-01-01
1       2006-01-01
2       2006-01-01
3       2006-01-01
4       2006-01-01
5       2006-01-01
6       2006-01-01
7       2006-01-01
8       2006-01-01
9       2006-01-01
10      2006-01-01
11      2006-01-01
12      2006-01-01
13      2006-01-01
14      2006-01-01
15      2006-01-01
16      2006-01-01
17      2006-01-01
18      2006-01-01
19      2006-01-01
20      2006-01-01
21      2006-01-01
22      2006-01-01
23      2006-01-01
24      2006-01-01
25      2006-01-01
26      2006-01-01
27      2006-01-01
28      2006-01-01
29      2006-01-01
           ...    
4431    2006-01-01
4432    2006-01-01
4433    2006-01-01
4434    2006-01-01
4435    2006-01-01
4436    2006-01-01
4437    2006-01-01
4438    2006-01-01
4439    2006-01-01
4440    2006-01-01
4441    2006-01-01
4442    2006-01-01
4443    2006-01-01
4444    2006-01-01
4445    2006-01-01
4446    2006-01-01
4447    2006-01-01
4448    2006-01-01
4449    2006-01-01
4450  

In [112]:
def split_date(fileData):
    splitter = fileData['date'].apply(lambda x: x.split('-'))
    fileData['year'] = splitter.apply(lambda x: x[0])
    fileData['month'] = splitter.apply(lambda x: x[1])
    fileData['dayOfMonth'] = splitter.apply(lambda x: x[2])
    return fileData
    

In [111]:
def true_extention(fileData):
    splitter = fileData['extention'].apply(lambda x: x.split('.'))
    try:
        fileData['file_name'] = splitter.apply(lambda x: x[0])
        fileData['true_extention'] = splitter.apply(lambda x: x[1])
        
    except:
        pass
    return fileData

In [177]:
def clean_browser(fileData, mode_of_browser):
    mode_of_browser = fileData['browser'].mode()
    fileData['browser'].fillna(mode_of_browser[0], inplace=True)
    return fileData